# **Harry Potter & the philosopher's Stone - CHATBOT**

In the Below code, I am installing I am ingesting and chunking diverse documents (like PDFs and EPUBs) using libraries such as unstructured and ebooklib.

I installed `langchain`, `groq`, `faiss-cpu`, `sentence-transformers`, `unstructured`, and `ebooklib` to build a system that loads and processes documents, embeds their content for semantic search, indexes them with FAISS, and uses a language model for intelligent question-answering.


Note:
Grok API: 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

In [ ]:
!pip install -q langchain groq faiss-cpu sentence-transformers unstructured ebooklib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.6/212.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 7.6 MB/s eta 0:00:00


Here, I’m installing `langchain-groq`, which is the integration that lets me connect **LangChain** with **Groq’s** lightning-fast language models for generating responses in my AI applications.

In [ ]:
pip install -U langchain-groq

Here, I’m installing `langchain-community`, which gives me access to a wide range of **community-maintained integrations, tools, and document loaders** that I can use within my LangChain applications.

In [ ]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

# To get the filename:
pdf_file = list(uploaded.keys())[0]

print("Uploaded file:", pdf_file)


Saving HP1-Sorcerers_Stone.pdf to HP1-Sorcerers_Stone.pdf
Uploaded file: HP1-Sorcerers_Stone.pdf


Note: Please upload the provided file to code. HP1-Sorcerers_Stone.pdf

1. **Library Installation**: Installed langchain, groq, faiss-cpu, sentence-transformers, and pypdf to enable document loading, text embedding, vector storage, and language model communication.

2. **API Key Setup:** Used getpass to securely enter and set the GROQ_API_KEY environment variable to access Groq's large language models.

3. **PDF Document Loading:**Loaded the PDF document using PyPDFLoader which extracts the text content from the PDF file.

4. **Text Splitting**: Used RecursiveCharacterTextSplitter to break the long document into smaller, overlapping chunks to make them suitable for embedding and retrieval.

5. **Embeddings Creation and Vector Store**: Generated numerical embeddings of the text chunks using HuggingFaceEmbeddings with the model "all-MiniLM-L6-v2" and stored them in a FAISS vector database for efficient semantic search.

6. **Language Model Initialization:** Initialized the Groq language model llama-3.3-70b-versatile with a specific temperature setting to control response creativity.

7. **RetrievalQA Chain Creation:** Built a RetrievalQA chain that first searches for relevant document chunks using FAISS and then passes them to the language model to generate contextually accurate answers.

In [ ]:

!pip install -q langchain groq faiss-cpu sentence-transformers pypdf

from getpass import getpass
import os
os.environ["GROQ_API_KEY"] = getpass("Paste Groq key: ")


from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/HP1-Sorcerers_Stone.pdf")
data = loader.load()


from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
chunks = text_splitter.split_documents(data)


from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.from_documents(chunks, embeddings)

\
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.7)


from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

Paste Groq key: ··········


This code creates an interactive chatbot where I can ask questions about the loaded PDF, and the system retrieves relevant information using the FAISS index and generates AI-based answers through the Groq language model.

In [ ]:

def interactive_chat():
    print("""
    ======================================
    Harry Potter QA System
    Type your question or 'quit' to exit
    ======================================
    """)

    while True:
        try:
            query = input("\nYour question: ").strip()

            if query.lower() in ['quit', 'exit', 'q']:
                print("\nGoodbye! Thanks for using the Harry Potter QA system!")
                break

            if not query:
                print("Please enter a question.")
                continue

            print("\nThinking...")
            result = qa_chain.invoke({"query": query})
            print(f"\nAnswer: {result['result']}")

        except KeyboardInterrupt:
            print("\nOperation cancelled by user.")
            break
        except Exception as e:
            print(f"\nSorry, an error occurred: {str(e)}")

interactive_chat()


    Harry Potter QA System
    Type your question or 'quit' to exit
    

Thinking...

Answer: Harry Potter is the main character in the book "Harry Potter and the Philosopher's Stone". He is a young boy who is a witch and a student at Hogwarts School. His parents, who were also witches and wizards, are dead. He is famous in the wizarding world because of something that happened to him when he was a baby, which is not fully explained in the provided text.

Thinking...

Answer: The four houses at Hogwarts are:

1. Gryffindor
2. Hufflepuff
3. Ravenclaw
4. Slytherin

Your question: who is harry potter?

Thinking...

Answer: Harry Potter is the main character in the book "Harry Potter and the Philosopher's Stone". He is a young boy who is a witch and a student at Hogwarts. His parents, who were also witches and wizards, are dead. He is famous in the wizarding world because of something that happened to him when he was a baby, which is not fully explained in the provided context.


**Summary:** In conclusion, this setup demonstrates how to build a powerful question-answering system by combining document processing, semantic search, and advanced language models. By using LangChain, FAISS, Hugging Face embeddings, and Groq’s LLM, I can efficiently retrieve relevant information from documents and generate accurate, context-based answers in an interactive chatbot. This approach can be applied to automate knowledge retrieval from any large set of documents or files.
